In [ ]:
import mysql.connector
import pandas as pd

# Connect to the database
cnx = mysql.connector.connect(
    host="XXX",
    user="XXX",
    password="XXX",
    database="XXX"
)

# Create a cursor to execute SQL statements
cursor = cnx.cursor()

# Execute a SELECT statement to fetch data from the table
query = "SELECT * FROM reddit_posts"
cursor.execute(query)

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Get the column names from the cursor description
columns = [column[0] for column in cursor.description]

# Convert the rows to a dataframe with the specified column names
df = pd.DataFrame(rows, columns=columns)
# Close the cursor and the database connection
cursor.close()
cnx.close()

In [ ]:

post_titles = np.array(df['post_title'])



In [ ]:
# tokenizing post tiles and counting words

import nltk
import re
nltk.download('punkt')
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import numpy as np

stop_words = set(stopwords.words("english"))
all_words = []

for title in post_titles:
    words = word_tokenize(title)
    words = [word.lower() for word in words if word.lower() not in stop_words]
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    words = [word for word in words if word]
    all_words += words

frequency_dist = FreqDist(all_words)

for word, frequency in frequency_dist.most_common():
    print(f"{word}: {frequency}")

In [ ]:
post_comments = np.concatenate((df['top_comment_one'], df['top_comment_two'], df['top_comment_three']))
post_comments = [comment for comment in post_comments if comment != '0']

In [ ]:
# tokenizing post comments

import nltk
import re
nltk.download('punkt')
nltk.download('stopwords') 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import numpy as np

stop_words = set(stopwords.words("english"))
all_words_comments = []

for comment in post_comments:
    comm = word_tokenize(comment)
    comm = [c.lower() for c in comm if c.lower() not in stop_words]
    comm = [re.sub(r'[^\w\s]', '', c) for c in comm]
    comm = [c for c in comm if c]
    all_words_comments += comm

frequency_dist = FreqDist(all_words_comments)

for c, frequency in frequency_dist.most_common():
    print(f"{c}: {frequency}")

In [ ]:
# now let's build a simple lda model to search for topics and compare them to linked descriptions in the future
post_bodies = df['post_body']

#gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
  
#spacy
import spacy

#vis
import pyLDAvis 
import pyLDAvis.gensim

stopwords = stopwords.words('english')

In [ ]:
def lemmatization(texts, allowed_tags = ['NOUN','ADJ','VERB','ADV']):
    nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
    texts_out = []
    for text in texts:
        if not isinstance(text, str):
            print(f"Found non-string element: {text}")
        else:
            tokens = nlp(text)
            lemmas = [token.lemma_ for token in tokens if token.pos_ in allowed_tags]
            texts_out.append(" ".join(lemmas))
    return texts_out

lemmatized_texts = lemmatization(post_bodies)

In [ ]:
import gensim 

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text)
        final.append(new)
    return final

data_words = gen_words(lemmatized_texts)

In [ ]:
id2word = corpora.Dictionary(data_words)

corpus = []

for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [ ]:
# creating the model 

lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 50,
                                            update_every = 1,
                                            chunksize = 50,
                                            passes = 10,
                                            alpha = 'auto')

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')

In [ ]:
# now i am gonna create bigrams/ trigrams as well as remove frequent words that do not bring much to my analysis

from gensim.models.phrases import Phrases, Phraser

# create bigram phrases
bigram_phrases = Phrases(data_words, min_count=5, threshold=80)

# transform the corpus using the bigram phrases
data_bigrams = [bigram_phrases[doc] for doc in data_words]

# create trigram phrases
trigram_phrases = Phrases(data_bigrams, min_count=5, threshold=80)

# transform the corpus using the trigram phrases
data_trigrams = [trigram_phrases[bigram_doc] for bigram_doc in data_bigrams]

# create a Phraser object for the trigram phrases (optional, but recommended for faster processing)
trigram_phraser = Phraser(trigram_phrases)

In [ ]:
# now let's remove the frequent unnecessary rows

from gensim.models import TfidfModel

# Create a dictionary of the trigrams in the data
id2word = corpora.Dictionary(data_trigrams)

# Convert the data to a bag-of-words corpus
texts = data_trigrams 
corpus = [id2word.doc2bow(text) for text in texts]

# Compute the TF-IDF scores for the corpus
tfidf = TfidfModel(corpus, id2word=id2word)

# Define a threshold for low-value words
low_value = 0.03

# Initialize lists for storing low-value words and missing words
words = []
words_missing_in_tfidf = []

# Loop over each document in the corpus
for i in range(0, len(corpus)):
    bow = corpus[i]
    
    # Initialize a list for storing low-value words
    low_value_words = []
    
    # Get the IDs of words with nonzero TF-IDF scores
    tfidf_ids = [id for id, value in tfidf[bow]]
    
    # Get the IDs of all words in the document
    bow_ids = [id for id, value in bow]
    
    # Find all words with a TF-IDF score below the threshold
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    
    # Add low-value words and missing words to the list of words to be dropped
    drops = low_value_words + words_missing_in_tfidf
    
    # Add dropped words to the list of words
    for item in drops:
        words.append(id2word[item])
    
    # Find all words that are missing from the TF-IDF model
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    # Remove all low-value and missing words from the document
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]  
    
    # Replace the original document with the cleaned-up version
    corpus[i] = new_bow

In [ ]:
# create new model

lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 50,
                                            update_every = 1,
                                            chunksize = 40,
                                            passes = 10,
                                            alpha = 'auto')

In [ ]:
#visualization 

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = 'mmds', R = 10)
vis
